# Performance

In this notebook we shall discuss the numerical performance of Kingdon.

In [24]:
import timeit
import numpy as np
from math import comb
import random

from kingdon import Algebra

In [25]:
alg = Algebra(3, 0, 1, numba=False)

We shall investigate the commutator product of a bivector with a vector; a common and important computation since this is the infinitesimal transformation of the vector under the transformation generated by the bivector.

We make a simple function that spits out an array of random values for the vector and bivector. (We make this into a function such that we can reuse it later.)

In [26]:
def generate_data(num_rows):
    bivector_shape = (comb(alg.d, 2), num_rows) if num_rows != 1 else comb(alg.d, 2)
    vector_shape = (comb(alg.d, 1), num_rows) if num_rows != 1 else comb(alg.d, 1)

    bvals = np.random.random(bivector_shape)
    uvals = np.random.random(vector_shape)
    return bvals, uvals

Let us first compute the commutator `b.cp(u)` for a 1-dimensional array.

In [27]:
bvals, uvals = generate_data(1)
b = alg.bivector(bvals)
u = alg.vector(uvals)

The first call will be expensive to make, because kingdon will have to generate the optimal code for this computation.

In [28]:
t = timeit.timeit('b.cp(u)', number=1, globals=globals())
print(f'Time to generate: {t:.2E}')

Time to generate: 1.06E-03


However, subsequent calls are significantly faster:

In [29]:
number = 10**5
t = timeit.timeit('b.cp(u)', number=number, globals=globals())
periter_python = t/number
print(f'Total time: {t}, time per iteration: {periter_python:.2E}')

Total time: 0.6242352909999909, time per iteration: 6.24E-06


The computation will of course slow down once we go for a two-dimensional arrays:

In [7]:
# bvals, uvals = generate_data(10000)
# b = alg.bivector(bvals)
# u = alg.vector(uvals)

In [8]:
# number = 10**4
# t = timeit.timeit('b.cp(u)', number=number, globals=globals())
# print(f'Total time: {t}, time per iteration: {t/number:.2E}')

Now let us do both of these scenarios again, but with numba enabled:

In [30]:
alg = Algebra(3, 0, 1, numba=True)

In [31]:
bvals, uvals = generate_data(1)
b = alg.bivector(bvals)
u = alg.vector(uvals)

In [32]:
timeit.timeit('b.cp(u)', number=1, globals=globals())

0.10953629200000137

In [33]:
number = 10**5
t = timeit.timeit('b.cp(u)', number=number, globals=globals())
periter_numba = t/number
print(f'Total time: {t}, time per iteration: {periter_numba:.2E}')

Total time: 0.4329873329999998, time per iteration: 4.33E-06


In [13]:
# bvals, uvals = generate_data(10000)
# b = alg.bivector(bvals)
# u = alg.vector(uvals)

In [14]:
# number = 10**4
# t = timeit.timeit('b.cp(u)', number=number, globals=globals())
# print(f'Total time: {t}, time per iteration: {t/number:.2E}')

The difference between numba and pure python becomes smaller as the size of the arrays increaces, because that means we spend more time in numpy and thus the lesser performance of Python becomes less relevant.

In [34]:
def generate_data(num_rows):
    bivector_shape = (2**alg.d, num_rows) if num_rows != 1 else 2**alg.d
    vector_shape = (2**alg.d, num_rows) if num_rows != 1 else 2**alg.d

    bvals = np.random.random(bivector_shape)
    uvals = np.random.random(vector_shape)
    return bvals, uvals

In [35]:
alg = Algebra(3, 0, 1, numba=False)

In [36]:
xvals, yvals = generate_data(1)
x = alg.multivector(xvals)
y = alg.multivector(yvals)

In [37]:
timeit.timeit('x.cp(y)', number=1, globals=globals())

0.007173792000003232

In [38]:
number = 10**5
t = timeit.timeit('x.cp(y)', number=number, globals=globals())
periter_python_full = t/number
print(f'Total time: {t}, time per iteration: {periter_python_full:.2E}')

Total time: 1.5837676669999894, time per iteration: 1.58E-05


In [39]:
periter_python_full/periter_python

2.5371325361353407

In [40]:
alg = Algebra(3, 0, 1, numba=True)

In [41]:
xvals, yvals = generate_data(1)
x = alg.multivector(xvals)
y = alg.multivector(yvals)

In [42]:
timeit.timeit('x.cp(y)', number=1, globals=globals())

0.32326487500000667

In [43]:
number = 10**5
t = timeit.timeit('x.cp(y)', number=number, globals=globals())
periter_numba_full = t/number
print(f'Total time: {t}, time per iteration: {periter_python_full:.2E}')

Total time: 0.45447316699997486, time per iteration: 1.58E-05


In [44]:
periter_numba_full/periter_numba

1.0496223153945594